##GAN&CLIP&GA - CIFAR 10 EVALUATION

Authors: Maciej Domagała, Adam Komorowski

### GENERATING IMAGES

In [1]:
#@title Initialization (Installation, Imports) - check github branch!
%%capture
import subprocess
import re

nvcc = subprocess.check_output(["nvcc", "--version"]).decode("utf-8")
version = re.findall("release (\d+\.\d+)", nvcc)[0]

pytorch_suffix = {
    "10.0": "+cu100",
    "10.1": "+cu101",
    "10.2": "",
}

pytorch_version = "1.7.1" + (pytorch_suffix[version] if version in pytorch_suffix else "+cu110")
torchvision_version = "0.8.2" + (pytorch_suffix[version] if version in pytorch_suffix else "+cu110")

!git clone https://github.com/maciejdomagala/clip_gans.git
%cd clip_gans
!git checkout evaluation ### WARNING!!!

try:
  import torch
except:
  !pip install torch=={pytorch_version} -f https://download.pytorch.org/whl/torch_stable.html

try:
  import torchvision
except:
  !pip install torchvision=={torchvision_version} -f https://download.pytorch.org/whl/torch_stable.html

!pip install pytorch_pretrained_biggan pymoo kornia ftfy tensorboard

# IMPORTS
import warnings

import argparse
import os
import torch
import numpy as np
import pickle
import math
from tqdm import tqdm_notebook
from pymoo.optimize import minimize
from pymoo.algorithms.so_genetic_algorithm import GA
from pymoo.factory import get_algorithm, get_decision_making, get_decomposition
from pymoo.visualization.scatter import Scatter
import torchvision
from IPython.display import Image, display
import urllib.request
import cv2

from config import get_config
from problem import GenerationProblem
from operators import get_operators


warnings.filterwarnings('ignore')

In [2]:
#@title Input Parameters (Code will produce (population_size*number_of_loops) evaluation images)
target = "deer" #@param ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
config = "DeepMindBigGAN256" #@param ["DeepMindBigGAN256", "DeepMindBigGAN512", "StyleGAN2_ffhq_nod", "StyleGAN2_car_nod", "StyleGAN2_church_nod"]
generations =  300#@param {type:"number"}
population_size = 64 #@param {type:"number"}
number_of_loops =  3#@param {type:"number"}
algorithm_param = 'ga' #@param ['ga', 'de']

In [3]:
#@title Config 
config = argparse.Namespace(
    config=config,
    target=target,
    device="cuda",
    generations=generations,
    tmp_folder="./results"
)
vars(config).update(get_config(config.config))
config.pop_size = population_size
config.algorithm = algorithm_param
config.generations = generations

if not os.path.exists(config.tmp_folder): os.mkdir(config.tmp_folder)
if not os.path.exists(f'{config.tmp_folder}/{config.target}'): os.mkdir(f'{config.tmp_folder}/{config.target}')


In [4]:
#@title Support methods

def split_into_tiles_and_save(image):
  """
  Method splits batch image from the Pymoo output to the equal tiles and save them
  to the results directory.
  """
  global config, iteration, loop, target
  M = int(image.shape[0]//math.sqrt(config.pop_size))
  N = int(image.shape[1]//math.sqrt(config.pop_size))
  tiles = [image[x:x+M,y:y+N] for x in range(0,image.shape[0],M) for y in range(0,image.shape[1],N)]
  if not os.path.exists(f'{config.tmp_folder}/{config.target}/{loop}'): os.mkdir(f'{config.tmp_folder}/{config.target}/{loop}')
  for j, each in enumerate(tiles):
    cv2.imwrite(f'./{config.tmp_folder}/{config.target}/{loop}/{target}_loop_{loop}_id_{j}.jpg', each)

def save_callback(algorithm):
    global config, iteration, loop, target

    iteration += 1

    sortedpop = sorted(algorithm.pop, key=lambda p: p.F)
    X = np.stack([p.X for p in sortedpop])      
    ls = config.latent(config)
    ls.set_from_population(X)
    if iteration == config.generations:
      with torch.no_grad():
        generated = algorithm.problem.generator.generate(ls)
        name = f'batch_loop_{loop}.jpg'
        algorithm.problem.generator.save(generated, f"./{config.tmp_folder}/{config.target}/{name}")
        im = cv2.imread(f"./{config.tmp_folder}/{config.target}/{name}")
        split_into_tiles_and_save(im)

def generate_image(loop: int):
  problem = GenerationProblem(config)
  operators = get_operators(config)

  algorithm = get_algorithm(
      config.algorithm,
      pop_size=config.pop_size,
      sampling=operators["sampling"],
      crossover=operators["crossover"],
      mutation=operators["mutation"],
      eliminate_duplicates=True,
      callback=save_callback,
      **(config.algorithm_args[config.algorithm] if "algorithm_args" in config and config.algorithm in config.algorithm_args else dict())
  )

  res = minimize(
      problem,
      algorithm,
      ("n_gen", config.generations),
      save_history=False,
      verbose=False,
  )

In [5]:
#@title RUN (GENERATE IMAGES)


for loop in tqdm_notebook(range(number_of_loops)):
  iteration=0
  generate_image(loop)

### CIFAR10 - CLASS EVALUATION

In [6]:
#@title Initialization
%%capture

%cd /content
!git clone https://github.com/vrakesh/CIFAR-10-Classifier
%cd CIFAR-10-Classifier/

from cifar_classifier import cifar_classifier
cc = cifar_classifier()
model = cc.get_model('cnn')
model.load_weights('cifar10.model.cnn.hdf5')
cifar10_labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [7]:
#@title RUN - EVALUATE CLASS
class_to_check = 'deer' #@param {type:"string"}
import PIL
folders_to_check = [dI for dI in os.listdir(f'/content/clip_gans/{config.tmp_folder}/{class_to_check}') if os.path.isdir(os.path.join(f'/content/clip_gans/{config.tmp_folder}/{class_to_check}', dI))]
list_of_images = list()
for folder in folders_to_check:
  for (dirpath, dirnames, filenames) in os.walk(f'/content/clip_gans/{config.tmp_folder[2:]}/{class_to_check}/{folder}'):
      list_of_images += [os.path.join(dirpath, file) for file in filenames]

acc_count = 0
for image_path in tqdm_notebook(list_of_images):
  im = (PIL.Image.open(image_path))
  im = im.resize((32, 32))
  im = np.expand_dims(im, 0)
  y_hat = model.predict(im)
  predicted_label = cifar10_labels[np.argmax(y_hat[0])]
  if predicted_label==class_to_check:
    acc_count += 1
print(f'\nAccuracy for {class_to_check.upper()} class: {round(acc_count/len(list_of_images)*100, 2)}%')
print(f'Number of evaluated images: {len(list_of_images)}')
print("")
print(f'Generation process summary:')
print(f'- Number of iterations per loop: {config.generations}')



Accuracy for DEER class: 4.94%
Number of evaluated images: 162

Generation process summary:
- Number of iterations per loop: 10


###CIFAR10 - GENERATE & EVALUATE ALL CLASSES

In [ ]:
!rm -r /content/clip_gans/results/* # uncomment if necessary

In [ ]:
#@title RUN ALL CIFAR10 CLASSES EVALUATION 

config = "DeepMindBigGAN256" #@param ["DeepMindBigGAN256", "DeepMindBigGAN512", "StyleGAN2_ffhq_nod", "StyleGAN2_car_nod", "StyleGAN2_church_nod"]
generations =  100#@param {type:"number"}
population_size = 64 #@param {type:"number"}
number_of_loops =  1#@param {type:"number"}
algorithm_param = 'ga' #@param ['ga', 'de']

%cd /content/clip_gans

#@title Config 
config = argparse.Namespace(
    config=config,
    target=target,
    device="cuda",
    generations=generations,
    tmp_folder="./results"
)
vars(config).update(get_config(config.config))
config.pop_size = population_size
config.algorithm = algorithm_param
config.generations = generations

if not os.path.exists(config.tmp_folder): os.mkdir(config.tmp_folder)

for target in cifar10_labels:
  print(f'\n{target} started.')
  config.target = target
  if not os.path.exists(f'{config.tmp_folder}/{config.target}'): os.mkdir(f'{config.tmp_folder}/{config.target}')
  for loop in range(number_of_loops):
    iteration=0
    generate_image(loop)
  print(f'{target} finished.')

print("")

for class_to_check in cifar10_labels:
  folders_to_check = [dI for dI in os.listdir(f'/content/clip_gans/{config.tmp_folder}/{class_to_check}') if os.path.isdir(os.path.join(f'/content/clip_gans/{config.tmp_folder}/{class_to_check}', dI))]
  list_of_images = list()
  for folder in folders_to_check:
    for (dirpath, dirnames, filenames) in os.walk(f'/content/clip_gans/{config.tmp_folder[2:]}/{class_to_check}/{folder}'):
        list_of_images += [os.path.join(dirpath, file) for file in filenames]

  acc_count = 0
  for image_path in list_of_images:
    im = (PIL.Image.open(image_path))
    im = im.resize((32, 32))
    im = np.expand_dims(im, 0)
    y_hat = model.predict(im)
    predicted_label = cifar10_labels[np.argmax(y_hat[0])]
    if predicted_label==class_to_check:
      acc_count += 1
  print(f'{class_to_check.upper()} accuracy: {round(acc_count/len(list_of_images)*100, 2)}%, ({acc_count}/{len(list_of_images)})')

/content/clip_gans

airplane started.
airplane finished.

automobile started.
automobile finished.

bird started.
bird finished.

cat started.
cat finished.

deer started.
deer finished.

dog started.
dog finished.

frog started.
frog finished.

horse started.
horse finished.

ship started.
ship finished.

truck started.
truck finished.

AIRPLANE accuracy: 0.0%, (0/81)
AUTOMOBILE accuracy: 7.41%, (6/81)
BIRD accuracy: 0.0%, (0/81)
CAT accuracy: 0.0%, (0/81)
DEER accuracy: 0.0%, (0/81)
DOG accuracy: 0.0%, (0/81)
FROG accuracy: 0.0%, (0/81)
HORSE accuracy: 0.0%, (0/81)
SHIP accuracy: 9.88%, (8/81)
TRUCK accuracy: 6.17%, (5/81)
